written by Yodai Takei and Jonathan White

In [1]:
import pandas as pd
import numpy as np
import copy
from copy import deepcopy

# Our numerical workhorses
import numpy as np
import pandas as pd
import re
from collections import defaultdict, Counter
import math
import scipy.optimize
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import scipy as sp
import scipy.stats
from scipy.spatial import distance
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette
import os

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Plotting modules
import matplotlib.pyplot as plt
import matplotlib.patches

# Package to perform PCA
import sklearn.datasets
import sklearn.decomposition
from sklearn.manifold import TSNE
from scipy.stats import zscore

# This is to enable inline displays for the purposes of the tutorial
%matplotlib inline

# This enables SVG graphics inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn makes plots look nice
import seaborn as sns



rc = {'lines.linewidth': 2, 'axes.labelsize': 18,  'axes.titlesize': 18, 'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('white')

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
#load codebook
cb_path = 'I:/OneDrive - California Institute of Technology/probe_design/output/'
cb = pd.read_csv(cb_path+'2021-12-11-NewBalance-loci-paint-barcodes-onoff.csv')
cb = cb[['name', 'chrom', 'hyb', 'channel', 'Round 1', 'Round 2', 'Round 3', 'Round 4']]

In [3]:
path = 'H:/Yodai/100k/2021-03-30-E14-100k-DNAfull-rep2/analysis/2021-06-27-E14-100k-rep2-final/paint decoding/'

max_thresh = 0.5 # new default is 0.5
min_thresh = 0.0 # default is 0

paint_round = [1,2,3,4]
paint_hyb = [1,2,3,4,5,6,7,8,9]
new_col_index = pd.MultiIndex.from_product([paint_round, paint_hyb], names = ['Round', 'paint_hyb'])


for pos in range(11):
    # load data
    df_1 = pd.read_csv(path+'2021-06-27-E14-100k-rep2-final-output-finalpoints-paints-pos'+str(pos)+'-ch1.csv')
    df_2 = pd.read_csv(path+'2021-06-27-E14-100k-rep2-final-output-finalpoints-paints-pos'+str(pos)+'-ch2.csv')
    df_3 = pd.read_csv(path+'2021-06-27-E14-100k-rep2-final-output-finalpoints-paints-pos'+str(pos)+'-ch3.csv')
    df = pd.concat([df_1,df_2,df_3],axis=0).reset_index(drop=True)
    df.set_index(["fov", "hyb", "channel", "cellID", "x", "y", "z", "dot_int"], inplace=True)

    
    df.columns = new_col_index
    df.sort_index(inplace= True)

    df_stacked_round = df.stack("Round")

    round_maxes = df_stacked_round.max("columns")
    round_mins = df_stacked_round.min("columns")
    round_maxes = round_maxes.unstack("Round")
    round_mins = round_mins.unstack("Round")
    dot_min_round_max = round_maxes.min("columns")
    dot_max_round_min = round_mins.max("columns")

    acceptable = dot_min_round_max.loc[np.logical_and(np.array(dot_min_round_max) > max_thresh, np.array(dot_max_round_min < min_thresh))]
    secondary = dot_min_round_max.drop(acceptable.index)

    filtered_df = df.loc[acceptable.index]
    secondary_df = df.loc[secondary.index]

    filtered_df = filtered_df.stack("Round")
    #secondary_df = secondary_df.stack("Round")

    filtered_df = filtered_df.idxmax("columns")

    filtered_df = filtered_df.unstack("Round")

    filtered_df.columns = ["Round 1", "Round 2", "Round 3", "Round 4"]

    max_dcd = pd.merge(filtered_df.reset_index(), cb, on=['hyb', 'channel', 'Round 1', 'Round 2', 'Round 3', 'Round 4'])

    max_dcd.to_csv('output/LC1-100k-001-012-2021-07-06-E14-100k-rep2-output-finalpoints-paints-decoded-onoff-chAll-pos'+str(pos)+'.csv',index=False)